Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

<img src="img/cnn_seq.png" width="300">

In [1]:
import pos
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

In [2]:
params = {
    'seq_len': 20,
    'batch_size': 128,
    'hidden_dim': 128,
    'kernel_sizes': [3, 5],
    'text_iter_step': 1,
    'lr': {'start': 5e-3, 'end': 5e-4},
    'n_epoch': 1,
    'display_step': 50,
}

In [3]:
def to_test_seq(*args):
    return [np.reshape(x[:(len(x)-len(x)%params['seq_len'])],
        [-1,params['seq_len']]) for x in args]

def iter_seq(x):
    return np.array([x[i: i+params['seq_len']] for i in range(
        0, len(x)-params['seq_len'], params['text_iter_step'])])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

In [4]:
def pipeline_train(X, y, sess):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(len(X)).batch(params['batch_size'])
    iterator = dataset.make_initializable_iterator()
    X_ph = tf.placeholder(tf.int32, [None, params['seq_len']])
    y_ph = tf.placeholder(tf.int32, [None, params['seq_len']])
    init_dict = {X_ph: X, y_ph: y}
    sess.run(iterator.initializer, init_dict)
    return iterator, init_dict

def pipeline_test(X, sess):
    dataset = tf.data.Dataset.from_tensor_slices(X)
    dataset = dataset.batch(params['batch_size'])
    iterator = dataset.make_initializable_iterator()
    X_ph = tf.placeholder(tf.int32, [None, params['seq_len']])
    init_dict = {X_ph: X}
    sess.run(iterator.initializer, init_dict)
    return iterator, init_dict


x_train, y_train, x_test, y_test, params['vocab_size'], params['n_class'], word2idx, tag2idx = pos.load_data()
X_train, Y_train = to_train_seq(x_train, y_train)
X_test, Y_test = to_test_seq(x_test, y_test)

sess = tf.Session()
params['lr']['steps'] = len(X_train) // params['batch_size']

iter_train, init_dict_train = pipeline_train(X_train, Y_train, sess)
iter_test, init_dict_test = pipeline_test(X_test, sess)

Vocab Size: 19124 | x_train: 211727 | x_test: 47377


In [5]:
def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, params['vocab_size'], params['hidden_dim'])
        x = tf.layers.dropout(x, 0.1, training=is_training)
        
        pad = tf.zeros([tf.shape(x)[0], 1, params['hidden_dim']])
        for k_sz in params['kernel_sizes']:
            n = (k_sz - 1) // 2
            _x = tf.concat([pad]*n + [x] + [pad]*n, 1)
            x += tf.layers.conv1d(_x, params['hidden_dim'], k_sz, activation=tf.nn.relu)

        logits = tf.layers.dense(x, params['n_class'])
    return logits

In [6]:
ops = {}

X_train_batch, y_train_batch = iter_train.get_next()
X_test_batch = iter_test.get_next()

logits_tr = forward(X_train_batch, reuse=False, is_training=True)
logits_te = forward(X_test_batch, reuse=True, is_training=False)

log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(
    logits_tr, y_train_batch, tf.count_nonzero(X_train_batch, 1))

ops['loss'] = tf.reduce_mean(-log_likelihood)

ops['global_step'] = tf.Variable(0, trainable=False)

ops['lr'] = tf.train.exponential_decay(
    params['lr']['start'], ops['global_step'], params['lr']['steps'],
    params['lr']['end']/params['lr']['start'])

ops['train'] = tf.train.AdamOptimizer(ops['lr']).minimize(
    ops['loss'], global_step=ops['global_step'])

ops['crf_decode'] = tf.contrib.crf.crf_decode(
    logits_te, trans_params, tf.count_nonzero(X_test_batch, 1))[0]

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
sess.run(tf.global_variables_initializer())
for epoch in range(1, params['n_epoch']+1):
    while True:
        try:
            _, step, loss, lr = sess.run([ops['train'],
                                          ops['global_step'],
                                          ops['loss'],
                                          ops['lr']])
        except tf.errors.OutOfRangeError:
            break
        else:
            if step % params['display_step'] == 0 or step == 1:
                print("Epoch %d | Step %d | Loss %.3f | LR: %.4f" % (epoch, step, loss, lr))
    
    Y_pred = []
    while True:
        try:
            Y_pred.append(sess.run(ops['crf_decode']))
        except tf.errors.OutOfRangeError:
            break
    Y_pred = np.concatenate(Y_pred)
    
    if epoch != params['n_epoch']:
        sess.run(iter_train.initializer, init_dict_train)
        sess.run(iter_test.initializer, init_dict_test)

Epoch 1 | Step 1 | Loss 74.740 | LR: 0.0050
Epoch 1 | Step 50 | Loss 3.259 | LR: 0.0047
Epoch 1 | Step 100 | Loss 1.595 | LR: 0.0043
Epoch 1 | Step 150 | Loss 0.938 | LR: 0.0041
Epoch 1 | Step 200 | Loss 0.494 | LR: 0.0038
Epoch 1 | Step 250 | Loss 0.300 | LR: 0.0035
Epoch 1 | Step 300 | Loss 0.412 | LR: 0.0033
Epoch 1 | Step 350 | Loss 0.269 | LR: 0.0031
Epoch 1 | Step 400 | Loss 0.187 | LR: 0.0029
Epoch 1 | Step 450 | Loss 0.208 | LR: 0.0027
Epoch 1 | Step 500 | Loss 0.174 | LR: 0.0025
Epoch 1 | Step 550 | Loss 0.138 | LR: 0.0023
Epoch 1 | Step 600 | Loss 0.157 | LR: 0.0022
Epoch 1 | Step 650 | Loss 0.076 | LR: 0.0020
Epoch 1 | Step 700 | Loss 0.242 | LR: 0.0019
Epoch 1 | Step 750 | Loss 0.157 | LR: 0.0018
Epoch 1 | Step 800 | Loss 0.146 | LR: 0.0016
Epoch 1 | Step 850 | Loss 0.131 | LR: 0.0015
Epoch 1 | Step 900 | Loss 0.099 | LR: 0.0014
Epoch 1 | Step 950 | Loss 0.131 | LR: 0.0013
Epoch 1 | Step 1000 | Loss 0.077 | LR: 0.0012
Epoch 1 | Step 1050 | Loss 0.125 | LR: 0.0012
Epoch 1 | 

In [8]:
print(classification_report(Y_test.ravel(), Y_pred.ravel(), target_names=tag2idx.keys()))

sample = ['I', 'love', 'you']
x = np.atleast_2d([word2idx[w] for w in sample] + [0]*(params['seq_len']-len(sample)))

ph = tf.placeholder(tf.int32, [None, params['seq_len']])
logits = forward(ph, reuse=True, is_training=False)
infer_op = tf.contrib.crf.crf_decode(logits, trans_params, tf.count_nonzero(ph, 1))[0]
idx2tag = {idx : tag for tag, idx in tag2idx.items()}

x = sess.run(infer_op, {ph: x})[0][:len(sample)]
print(' '.join(sample))
print(' '.join([idx2tag[idx] for idx in x if idx != 0]))

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 43, does not match size of target_names, 45
  .format(len(labels), len(target_names))


             precision    recall  f1-score   support

      <pad>       0.89      0.95      0.92      6639
         NN       0.99      1.00      0.99      5070
         IN       1.00      1.00      1.00      4020
         DT       0.96      0.93      0.95       912
        VBZ       0.97      0.94      0.96      1354
         RB       0.91      0.90      0.90      1103
        VBN       1.00      1.00      1.00      1177
         TO       0.92      0.94      0.93      1269
         VB       0.83      0.94      0.88      2962
         JJ       0.97      0.89      0.93      3034
        NNS       0.97      0.89      0.93      4803
        NNP       1.00      1.00      1.00      2389
          ,       1.00      1.00      1.00      1214
         CC       1.00      1.00      1.00       433
        POS       1.00      1.00      1.00      1974
          .       0.90      0.92      0.91       539
        VBP       0.92      0.87      0.89       727
        VBG       1.00      1.00      1.00   